# 科学网会议数据检索

科学网会议网站为网民提供了权威的学术会议信息。利用科学网会议网站进行数据检索，可以按照输入的关键词进行会议检索，也可以按照召开时间进行会议检索，也可以按照会议的学科分类进行会议检索。

入口地址是"http://meeting.sciencenet.cn/"。 针对3种不同形式的检索需求，可以分别用3个不同的函数来实现。先给出基本的框架：

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
import pandas as pd

#获得输入url的页面内容
def getHTMLText(url):
    return ''

#获得会议的名称、地点、召开时间
def getMeetingInfo(url,lst):
    pass

#按照用户输入的关键字进行数据检索
def searchByKeyword(pageNum,lst):
    pass
    
#按照用户选择的年份和月份进行数据检索        
def searchByMonth(pageNum,lst):
    pass

#按照用户选择的分类进行数据检索
def searchByClass(pageNum,lst):
    pass

def main():
    pageNumber=2    #控制读取的页数
    meetingInfos=[]
    select=input('请选择会议的检索方式：\n1.按输入的关键词检索\n2.按召开月历检索\n3.按会议分类检索\n')
    if select==str(1):
        searchByKeyword(pageNumber,meetingInfos)
    
    elif select==str(2):
        searchByMonth(pageNumber,meetingInfos)
    
    else:
        searchByClass(pageNumber,meetingInfos)
    
main()

对于按照输入的关键词进行会议检索，我们以输入“人工智能”为例，跳转到的网页的URL为"http://meeting.sciencenet.cn/index.php?s=%2FCategory%2Fsearchmeeting&keyword=%E4%BA%BA%E5%B7%A5%E6%99%BA%E8%83%BD&p=1"。
可以看出，keyword的值就是输入的关键词编码后的形式，p的值就是当前的页码，第一页就是p=1，第二页就是p=2，以此类推。

因此，searchByKeyword函数可以定义为：

In [ ]:
def searchByKeyword(pageNum,lst):
    keyword=input('请输入关键字进行会议搜索：')
    for i in range(pageNum):
        quary_url='http://meeting.sciencenet.cn/'+'index.php?s=%2FCategory%2Fsearchmeeting&keyword='+quote(keyword)+'&p='+str(i+1)
        getMeetingInfo(quary_url,lst)

对于按照召开时间进行会议检索，科学网会议网站上提供的是按照月份进行筛选。我们以2019年7月为例，跳转到的网页的URL为"http://meeting.sciencenet.cn/index.php?s=%2FCategory%2Fmth_meeting&mth=2019-07&p=1"。
可以看出，mth的值就是按照月历进行筛选的筛选条件，因此我们可以按照用户输入的值来获得不同月份的网页的URL，p的值依然是当前的页码。

因此，searchByMonth函数可以定义为：

In [ ]:
def searchByMonth(pageNum,lst):
    year=input('请输入检索年份：')
    month=input('请输入检索月份：')
    month=(month if len(month)==2 else '0'+month)    #按照URL中的格式，月份的表示如7月，必须表示为07而不能是7
    for i in range(pageNum):
        quary_url='http://meeting.sciencenet.cn/'+'index.php?s=%2FCategory%2Fmth_meeting&mth='+year+'-'+month+'&p='+str(i+1)
        getMeetingInfo(quary_url,lst

对于按照会议的学科类别进行会议检索，科学网会议网站上提供了8种不同的类别选择。我们以”生命科学”为例，跳转到的网页的URL为"http://meeting.sciencenet.cn/index.php?s=%2FCategory%2Findex&cid=2&p=1"。
可以看出，URL中用cid的值来区分不同的类别，对于8种不同的类别，cid的值分别是从2到9，p的值依然是当前的页码。

因此，searchByClass函数可以定义为：

In [1]:
#按照用户选择的分类进行数据检索
def searchByClass(pageNum,lst):
    url='http://meeting.sciencenet.cn/index.php?s=/Category/moreCategory'   #在这个页面上能更方便的获得会议的领域分类信息
    html=getHTMLText(url)
    soup=BeautifulSoup(html,'html.parser')
    
    classifies=[]       #按学科领域对会议进行分类，存储所有不同类别的名称
    for b in soup.find_all('b',attrs={'id':'nm'}):
        classifies.append(b.text)
    
    for i in range(len(classifies)):
        print(str(i+1)+'、'+classifies[i])
        
    choose=input('请选择会议分类：')
    
    for i in range(pageNum):
        quary_url='http://meeting.sciencenet.cn/'+'index.php?s=%2FCategory%2Findex&cid='+str(int(choose)+1)+'&p='+str(i+1)
        getMeetingInfo(quary_url,lst)

在按照不同的形式检索得到的页面中，会议名称、地点、召开时间都可以定位在相同的html代码段中，所以我们要定义一个可复用的函数getMeetingInfo。以其中一个会议的信息为例，可以定位在一个tr标签中。

其中，会议名称为具有属性{'width':'60%'}的td标签的字符串内容；地点为具有属性{'class':'STYLE2'}的第1个td标签的字符串内容；召开时间为具有属性{'class':'STYLE2'}的第2个td标签的字符串内容。

最后，可以把得到的数据存入一个excel文件中，可用于后续的数据分析。下面给出完整的代码实现：

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
import pandas as pd

#获得输入url的页面内容
def getHTMLText(url):
    try:
        r=requests.get(url)
        r.raise_for_status()
        r.encoding='utf-8'
        return r.text
    except:
        return ''    

#获得会议的名称、时间、地点
def getMeetingInfo(url,lst):
    html=getHTMLText(url)
    soup=BeautifulSoup(html,'html.parser')
    for tr in soup.find_all('tr'):
        infoDict={}
        try:
            infoDict['会议名称']=tr.find('td',attrs={'width':'60%'}).text.strip()
            infoDict['地点']=tr.find_all('td',attrs={'class':'STYLE2'})[0].text.strip()
            infoDict['召开时间']=tr.find_all('td',attrs={'class':'STYLE2'})[1].text.strip()
        except:
            continue
        lst.append(infoDict)

#按照用户输入的关键字进行数据检索
def searchByKeyword(pageNum,lst):
    keyword=input('请输入关键字进行会议搜索：')
    for i in range(pageNum):
        quary_url='http://meeting.sciencenet.cn/'+'index.php?s=%2FCategory%2Fsearchmeeting&keyword='+quote(keyword)+'&p='+str(i+1)
        getMeetingInfo(quary_url,lst)
        
#按照用户选择的年份和月份进行数据检索        
def searchByMonth(pageNum,lst):
    year=input('请输入检索年份：')
    month=input('请输入检索月份：')
    month=(''+month if len(month)==2 else '0'+month)
    for i in range(pageNum):
        quary_url='http://meeting.sciencenet.cn/'+'index.php?s=%2FCategory%2Fmth_meeting&mth='+year+'-'+month+'&p='+str(i+1)
        getMeetingInfo(quary_url,lst)
        
#按照用户选择的分类进行数据检索
def searchByClass(pageNum,lst):
    url='http://meeting.sciencenet.cn/index.php?s=/Category/moreCategory'
    html=getHTMLText(url)
    soup=BeautifulSoup(html,'html.parser')
    
    classifies=[]       #按学科领域对会议进行分类，存储所有不同类别的名称
    for b in soup.find_all('b',attrs={'id':'nm'}):
        classifies.append(b.text)
    
    for i in range(len(classifies)):
        print(str(i+1)+'、'+classifies[i])
        
    choose=input('请选择会议分类：')
    
    for i in range(pageNum):
        quary_url='http://meeting.sciencenet.cn/'+'index.php?s=%2FCategory%2Findex&cid='+str(int(choose)+1)+'&p='+str(i+1)
        getMeetingInfo(quary_url,lst)
        
def main():
    
    pageNumber=2
    meetingInfos=[]
    select=input('请选择会议的检索方式：\n1.按输入的关键词检索\n2.按召开月历检索\n3.按会议分类检索\n')
    if select==str(1):
        searchByKeyword(pageNumber,meetingInfos)
    
    elif select==str(2):
        searchByMonth(pageNumber,meetingInfos)
    
    else:
        searchByClass(pageNumber,meetingInfos)
    data=pd.DataFrame(meetingInfos)
    data.to_excel('E:/sciencenet_meeting.xlsx')    #把数据保存为excel文件
    
main()